In [21]:
import pandas as pd
import hopsworks
import joblib
import datetime
#import dataframe_image as df1
from datetime import datetime
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/39062
Connected. Call `.close()` to terminate connection gracefully.


In [22]:
mr = project.get_model_registry()
model = mr.get_model('bank_note', version=1)
model_dir = model.download()
model = joblib.load(model_dir + '/bank_note_model.pkl')

Connected. Call `.close()` to terminate connection gracefully.

In [23]:
feature_view = fs.get_feature_view(name='bank_notes_dataframe', version=1)

In [24]:
batch_data = feature_view.get_batch_data()
predictions = model.predict(batch_data)

2023-05-05 05:20:59,311 INFO: USE `bwhiz_featurestore`
2023-05-05 05:21:00,170 INFO: SELECT `fg0`.`variance` `variance`, `fg0`.`skewness` `skewness`, `fg0`.`curtosis` `curtosis`, `fg0`.`entropy` `entropy`
FROM `bwhiz_featurestore`.`bank_notes_dataframe_1` `fg0`


In [25]:
predictions

array([1, 1, 1, ..., 0, 1, 0])

In [26]:
batch_data

,variance,skewness,curtosis,entropy
0,0.224320,-0.521470,-0.403860,1.201700
1,-2.298700,-5.227000,5.630000,0.917220
2,-4.504600,-5.812600,10.886700,-0.528460
3,3.966300,10.168400,-4.113100,-4.605600
4,-2.280400,-0.306260,1.334700,1.376300
...,...,...,...,...
1345,0.379800,0.709800,0.757200,-0.444400
1346,2.467300,1.392600,1.712500,0.414210
1347,3.779100,2.576200,1.309800,0.565500
1348,-4.032270,-8.321182,6.125914,-5.591904


In [27]:
# prediciton on the last bank note sent :
latest_bank_note = predictions[predictions.size - 1]

In [28]:
bank_note_fg = fs.get_feature_group(name='bank_notes_dataframe', version=1)
df = bank_note_fg.read()
df

2023-05-05 05:21:05,827 INFO: USE `bwhiz_featurestore`
2023-05-05 05:21:06,601 INFO: SELECT `fg0`.`variance` `variance`, `fg0`.`skewness` `skewness`, `fg0`.`curtosis` `curtosis`, `fg0`.`entropy` `entropy`, `fg0`.`output` `output`
FROM `bwhiz_featurestore`.`bank_notes_dataframe_1` `fg0`


,variance,skewness,curtosis,entropy,output
0,0.224320,-0.521470,-0.403860,1.201700,1
1,-2.298700,-5.227000,5.630000,0.917220,1
2,-4.504600,-5.812600,10.886700,-0.528460,1
3,3.966300,10.168400,-4.113100,-4.605600,0
4,-2.280400,-0.306260,1.334700,1.376300,1
...,...,...,...,...,...
1345,0.379800,0.709800,0.757200,-0.444400,0
1346,2.467300,1.392600,1.712500,0.414210,0
1347,3.779100,2.576200,1.309800,0.565500,0
1348,-4.032270,-8.321182,6.125914,-5.591904,1


In [29]:
### getting the actual label of the last row of data:
label = df.iloc[-1]['output']
label

0.0

In [30]:
monitor_fg = fs.get_or_create_feature_group(name = 'bank_note_predictions',
                                           version = 1,
                                           primary_key = ['datetime'],
                                           description = 'bank note prediction/output Monitoring')

In [31]:
now = datetime.now().strftime("%m/%d/%y, %H:%M:%S")

data = {
    'prediction' : [latest_bank_note],
    'label' : [label],
    'datetime' :[now]
}

monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/39062/jobs/named/bank_note_predictions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1c8e0863ca0>, None)

In [32]:
history_df = monitor_fg.read()
history_df

2023-05-05 05:22:09,669 INFO: USE `bwhiz_featurestore`
2023-05-05 05:22:10,427 INFO: SELECT `fg0`.`prediction` `prediction`, `fg0`.`label` `label`, `fg0`.`datetime` `datetime`
FROM `bwhiz_featurestore`.`bank_note_predictions_1` `fg0`


,prediction,label,datetime
0,0,0.0,"05/04/23, 16:28:29"
1,0,0.0,"05/05/23, 05:21:10"


In [33]:
df_recent = history_df.tail(5)

In [34]:
#df1.export(df_recent, 'assets/df_recent.png') #save an image of the dataframe's last five output

In [35]:
predictions = history_df['prediction']
labels = history_df['label']

results = confusion_matrix(labels, predictions)
print(results)

[[2]]


In [36]:
try:
    if results.shape == (2,2):
        cm = sns.heatmap(results, annot=True)
        
        fig = cm.get_figure()
        fig.savefig('assets/confusion_matrix.png')
        results
    else:
        print('The confusion isnt a square matrix yet , run the feature pipeline with BACKFILL == False')
except:
    pass

The confusion isnt a square matrix yet , run the feature pipeline with BACKFILL == False
